In [64]:
import os
import time

import re
import requests
import warnings

import langchain
from langchain.llms import OpenAI
from langchain.agents import load_tools, AgentType, Tool, initialize_agent
os.environ["OPENAI_API_KEY"] = "OPEN_API_KEY"
from bs4 import BeautifulSoup

warnings.filterwarnings("ignore")

In [65]:

llm=OpenAI(temperature=0,
           model_name="gpt-3.5-turbo-16k-0613")

### Defining the custom Tools

In [66]:
pip install yfinance

In [67]:
import yfinance as yf



def get_stock_price(ticker,history=5):
    
    if "." in ticker:
        ticker=ticker.split(".")[0]
    ticker=ticker+".NS"
    stock = yf.Ticker(ticker)
    df = stock.history(period="1y")
    df=df[["Close","Volume"]]
    df.index=[str(x).split()[0] for x in list(df.index)]
    df.index.rename("Date",inplace=True)
    df=df[-history:]
    
    
    return df.to_string()

print(get_stock_price("TITAN"))

                  Close   Volume
Date                            
2024-05-13  3253.649902  1294603
2024-05-14  3288.899902   620163
2024-05-15  3268.550049   657036
2024-05-16  3333.899902  1404971
2024-05-17  3323.800049   196207


In [68]:


def google_query(search_term):
    if "news" not in search_term:
        search_term=search_term+" stock news"
    url=f"https://www.google.com/search?q={search_term}&cr=countryIN"
    url=re.sub(r"\s","+",url)
    return url

def get_recent_stock_news(company_name):

    headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36'}

    g_query=google_query(company_name)
    res=requests.get(g_query,headers=headers).text
    soup=BeautifulSoup(res,"html.parser")
    news=[]
    for n in soup.find_all("div","n0jPhd ynAwRc tNxQIb nDgy9d"):
        news.append(n.text)
    for n in soup.find_all("div","IJl0Z"):
        news.append(n.text)


    if len(news)>6:
        news=news[:4]
    else:
        news=news
    news_string=""
    for i,n in enumerate(news):
        news_string+=f"{i}. {n}\n"
    top5_news="Recent News:\n\n"+news_string
    
    return top5_news


print(get_recent_stock_news("Asian paints"))

Recent News:

0. Asian Paints Share Price Live blog for 17 May 2024 | Mint
1. Asian Paints Share Price Live Updates: Asian Paints Records 3-Month Return 
of -6.44% as Closing Price St...
2. Asian Paints Share Price Highlights : Asian Paints closed today at 
₹2814.25, up 0.05% from yesterday's ₹2812.95 | Mint



In [69]:


def get_financial_statements(ticker):
    
    if "." in ticker:
        ticker=ticker.split(".")[0]
    else:
        ticker=ticker
    ticker=ticker+".NS"    
    company = yf.Ticker(ticker)
    balance_sheet = company.balance_sheet
    if balance_sheet.shape[1]>=3:
        balance_sheet=balance_sheet.iloc[:,:3]   
    balance_sheet=balance_sheet.dropna(how="any")
    balance_sheet = balance_sheet.to_string()
    
    return balance_sheet
print(get_financial_statements("TATAPOWER.NS"))

                                                          2024-03-31       2023-03-31       2022-03-31
Ordinary Shares Number                                  3195339547.0     3196991847.0     3196991847.0
Share Issued                                            3195339547.0     3196991847.0     3196991847.0
Net Debt                                              461554700000.0   447847600000.0   445128500000.0
Total Debt                                            536894500000.0   529230900000.0   511952100000.0
Tangible Book Value                                   291385400000.0   255477800000.0   192170700000.0
Invested Capital                                      818351000000.0   777619500000.0   700316500000.0
Working Capital                                      -127861500000.0  -130013600000.0  -141013900000.0
Net Tangible Assets                                   291385400000.0   255477800000.0   192170700000.0
Capital Lease Obligations                              42096400000.0    3

### Custom tools

##### Approach 2

In [70]:
import openai

import os
os.environ["OPENAI_API_KEY"] = "OPEN_API_KEY"
openai.api_key="OPEN_API_KEY"


In [71]:
from langchain.llms import OpenAI

llm=OpenAI(temperature=0,
           model_name="gpt-3.5-turbo-16k-0613")

In [72]:

import json
function=[
        {
        "name": "get_company_Stock_ticker",
        "description": "This will get the indian NSE/BSE stock ticker of the company",
        "parameters": {
            "type": "object",
            "properties": {
                "ticker_symbol": {
                    "type": "string",
                    "description": "This is the stock symbol of the company.",
                },

                "company_name": {
                    "type": "string",
                    "description": "This is the name of the company given in query",
                }
            },
            "required": ["company_name","ticker_symbol"],
        },
    }
]



def get_stock_ticker(query):
    response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            temperature=0,
            messages=[{
                "role":"user",
                "content":f"Given the user request, what is the comapany name and the company stock ticker ?: {query}?"
            }],
            functions=function,
            function_call={"name": "get_company_Stock_ticker"},
    )
    message = response["choices"][0]["message"]
    arguments = json.loads(message["function_call"]["arguments"])
    company_name = arguments["company_name"]
    company_ticker = arguments["ticker_symbol"]
    return company_name,company_ticker

get_stock_ticker("How is ths stock of Paytm doing?")

('Paytm', 'PAYTM')

In [73]:
def Anazlyze_stock(query):
    Company_name, ticker = get_stock_ticker(query)
    print({"Query": query, "Company_name": Company_name, "Ticker": ticker})
    
    stock_data = get_stock_price(ticker, history=10)
    stock_financials = get_financial_statements(ticker)
    stock_news = get_recent_stock_news(Company_name)
    
    available_information = f"Stock Financials: {stock_financials}\n\nStock News: {stock_news}"

    print("\n\nAnalyzing.....\n")
    
    analysis = llm(
        f"Give detail stock analysis, Use the available data and provide investment recommendation. "
        f"The user is fully aware about the investment risk, don't include any kind of warning like "
        f"'It is recommended to conduct further research and analysis or consult with a financial advisor before making an investment decision' in the answer. "
        f"User question: {query}. "
        f"You have the following information available about {Company_name}. "
        f"Write (5-8) pointwise investment analysis to answer user query, At the end conclude with proper explanation. "
        f"Try to give positives and negatives: {available_information}"
    )
    
    print(analysis)

    return analysis


In [74]:

out=Anazlyze_stock("Shall I invest in Adani power right now?")    


{'Query': 'Shall I invest in Adani power right now?', 'Company_name': 'Adani Power', 'Ticker': 'ADANIPOWER.NS'}


Analyzing.....

1. Adani Power has shown positive financial growth over the past three years, with an increase in total assets from INR 819.81 billion in 2022 to INR 923.25 billion in 2024. This indicates a growing business and potential for future expansion.

2. The company's net tangible assets have also increased from INR 185.01 billion in 2022 to INR 429.42 billion in 2024, indicating a strong financial position and a solid foundation for growth.

3. Adani Power has a large amount of debt, with total debt of INR 487.44 billion in 2022 and INR 346.16 billion in 2024. This high level of debt may pose a risk to the company's financial stability and could impact its ability to generate profits in the future.

4. The stock has recently reached a lifetime high, indicating positive investor sentiment and potential for further growth. Experts predict a 25% upside in the stock p

In [75]:
Anazlyze_stock("Is it a good time to invest in Yes Bank?")    

{'Query': 'Is it a good time to invest in Yes Bank?', 'Company_name': 'Yes Bank', 'Ticker': 'YESBANK'}


Analyzing.....

1. Ordinary Shares Number: The number of ordinary shares of Yes Bank has remained relatively stable over the past three years, indicating no significant dilution or increase in the company's equity base.

2. Net Debt: Yes Bank's net debt has increased significantly from 13.29 billion in 2022 to 208.37 billion in 2023, and further to 611.89 billion in 2024. This indicates a higher level of debt burden on the company, which may impact its financial stability and ability to generate profits.

3. Tangible Book Value: The tangible book value of Yes Bank has shown a consistent increase over the past three years, from 336.99 billion in 2022 to 407.18 billion in 2023, and further to 421.55 billion in 2024. This indicates that the company's assets are growing and its financial position is improving.

4. Total Capitalization: Yes Bank's total capitalization has also shown a co

"1. Ordinary Shares Number: The number of ordinary shares of Yes Bank has remained relatively stable over the past three years, indicating no significant dilution or increase in the company's equity base.\n\n2. Net Debt: Yes Bank's net debt has increased significantly from 13.29 billion in 2022 to 208.37 billion in 2023, and further to 611.89 billion in 2024. This indicates a higher level of debt burden on the company, which may impact its financial stability and ability to generate profits.\n\n3. Tangible Book Value: The tangible book value of Yes Bank has shown a consistent increase over the past three years, from 336.99 billion in 2022 to 407.18 billion in 2023, and further to 421.55 billion in 2024. This indicates that the company's assets are growing and its financial position is improving.\n\n4. Total Capitalization: Yes Bank's total capitalization has also shown a consistent increase over the past three years, indicating that the company has been able to raise capital and streng

In [76]:
Anazlyze_stock("How vednata share is doing?")    

{'Query': 'How vednata share is doing?', 'Company_name': 'Vedanta', 'Ticker': 'VEDL'}


Analyzing.....

1. Vedanta has recently approved a fundraise of Rs 8,500 crore and declared an interim dividend of Rs 11 per share. This indicates that the company is taking steps to strengthen its financial position and provide returns to its shareholders.

2. The stock has witnessed a significant rise of 77% in the past two months. While this may be seen as a positive indicator of investor confidence, it also raises concerns about the stock being overbought and the potential for a correction in the near future.

3. The company's financials show a consistent increase in net debt over the past three years. This indicates that Vedanta has been relying on debt to finance its operations and investments. High levels of debt can pose a risk to the company's financial stability and ability to generate profits.

4. On the positive side, Vedanta's tangible book value has shown a consistent increase over the

"1. Vedanta has recently approved a fundraise of Rs 8,500 crore and declared an interim dividend of Rs 11 per share. This indicates that the company is taking steps to strengthen its financial position and provide returns to its shareholders.\n\n2. The stock has witnessed a significant rise of 77% in the past two months. While this may be seen as a positive indicator of investor confidence, it also raises concerns about the stock being overbought and the potential for a correction in the near future.\n\n3. The company's financials show a consistent increase in net debt over the past three years. This indicates that Vedanta has been relying on debt to finance its operations and investments. High levels of debt can pose a risk to the company's financial stability and ability to generate profits.\n\n4. On the positive side, Vedanta's tangible book value has shown a consistent increase over the past three years. This indicates that the company has been able to generate value from its tangi